In [24]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

label - 0 means news is real
label - 1 means news is fake

In [25]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [26]:
# printing the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-processing

In [27]:
# loading the dataset to a pandas DataFrame
news_dataset = pd.read_csv('/content/train.csv',engine='python', error_bad_lines=False)

<ipython-input-27-a34c38dca573>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  news_dataset = pd.read_csv('/content/train.csv',engine='python', error_bad_lines=False)
Skipping line 13535: field larger than field limit (131072)
Skipping line 19783: field larger than field limit (131072)
Skipping line 13541: Expected 5 fields in line 13541, saw 6
Skipping line 19782: Expected 5 fields in line 19782, saw 7
Skipping line 19783: Expected 5 fields in line 19783, saw 10
Skipping line 19785: Expected 5 fields in line 19785, saw 14
Skipping line 19787: Expected 5 fields in line 19787, saw 8
Skipping line 19789: Expected 5 fields in line 19789, saw 7
Skipping line 19790: Expected 5 fields in line 19790, saw 15
Skipping line 19791: Expected 5 fields in line 19791, saw 9
Skipping line 19792: Expected 5 fields in line 19792, saw 9
Skipping line 19793: Expected 5 fields in line 19793, saw 12
Skipping lin

In [28]:
news_dataset.shape

(20822, 5)

In [29]:
# print the first 5 rows of the dataframe
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [30]:
# counting the number of missing values in the dataset
news_dataset.isnull().sum()

id           0
title      565
author    1975
text        59
label       22
dtype: int64

In [31]:
# replacing the null values with empty string
news_dataset = news_dataset.fillna('')

In [32]:
# merging the author name and news title
news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']

In [33]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20817    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20818    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20819    Michael J. de la Merced and Rachel Abrams Macy...
20820    Alex Ansary NATO, Russia To Hold Parallel Exer...
20821              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20822, dtype: object


In [34]:
# separating the data & label
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']

In [35]:
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20817  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20818  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20819  20797  Macy’s Is Said to Receive Takeover Approach by...   
20820  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20821  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

Stemming:

Stemming is the process of reducing a word to its Root word

example:
actor, actress, acting --> act

In [36]:
port_stem = PorterStemmer()

In [37]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [38]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [39]:
print(news_dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20817    jerom hudson rapper trump poster child white s...
20818    benjamin hoffman n f l playoff schedul matchup...
20819    michael j de la merc rachel abram maci said re...
20820    alex ansari nato russia hold parallel exercis ...
20821                            david swanson keep f aliv
Name: content, Length: 20822, dtype: object


In [40]:
#separating the data and label
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [41]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [42]:
print(Y)

['1' '0' '1' ... '0' '1' '1']


In [43]:
Y.shape

(20822,)

In [44]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [45]:
print(X)

  (0, 15695)	0.2848972510323515
  (0, 13478)	0.2560149172431211
  (0, 8912)	0.36364344795259246
  (0, 8633)	0.29217179802324905
  (0, 7695)	0.24789859828097843
  (0, 7008)	0.21878792690522433
  (0, 4975)	0.2333632865672138
  (0, 3794)	0.2705797813233836
  (0, 3602)	0.3599409648981374
  (0, 2960)	0.24689141014750257
  (0, 2484)	0.3676990592743697
  (0, 267)	0.27014778053672167
  (1, 16807)	0.30071934786077886
  (1, 6819)	0.1904866118084521
  (1, 5506)	0.7143145691090199
  (1, 3570)	0.2637458518155929
  (1, 2814)	0.19096625123288233
  (1, 2224)	0.3827200188088641
  (1, 1894)	0.15524631265972175
  (1, 1497)	0.2939921887587299
  (2, 15620)	0.41544918282265714
  (2, 9623)	0.49350103437755793
  (2, 5971)	0.34747260973778105
  (2, 5392)	0.38665757315792654
  (2, 3104)	0.4609666076640373
  :	:
  (20819, 13126)	0.2482554055030498
  (20819, 12348)	0.2726336911024517
  (20819, 12142)	0.2477854180074755
  (20819, 10310)	0.08040873089640542
  (20819, 9591)	0.17456730094721287
  (20819, 9521)	0.2954

Splitting the dataset to training & test data

In [50]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2,random_state=2)

Training the Model: Logistic Regression

In [51]:
model = LogisticRegression()

In [52]:
model.fit(X_train, Y_train)

LogisticRegression()

Evaluation

accuracy score

In [53]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [54]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9924356126553401


In [55]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [56]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9815126050420168


Making a Predictive System

In [57]:
X_new = X_test[3]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

['0']
The news is Fake


In [58]:
print(Y_test[3])

0
